# Dimensionality Reduction in Python

In this lecture we will talk about two techniques that can be used to reduce the dimension of your data set: PCA and t-SNE.

## Why Dimensionality Reduction?

In a number of situations we would be fine without reducing the dimension, for instance consider our classic iris dataset.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

iris = sns.load_dataset("iris")

In [ ]:
sns.pairplot(iris, hue = "species")
plt.show()

With this data set the dimension is manageable. We are examining 2-dimensional slices of a 4-dimensional data set. We can wrap our 3-dimensional brains around that. However, let's consider the MNIST dataset.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

In [ ]:
print(digits.data.shape)

In [ ]:
plt.gray()
plt.matshow(digits.images[440])
plt.show()

This data set has many more dimensions, if we wanted to make a pairs plot of this data it would be 64 by 64 and essentially unintelligible. A data reduction could be incredibly useful here.

Dimensionality reduction is useful for both data analysis/visualization and prediction tasks. Let's dive in

## Principal Components Analysis (PCA)

This technique has been around since Karl Pearson's 1903 paper, <a href = "https://www.tandfonline.com/doi/abs/10.1080/14786440109462720"><i>On lines and planes of closest fit to systems of points in space</i></a>. It is well understood and, because of that, very popular. We'll first cover the idea behind PCA, then give a brief outline of its mathematical formulation.

### What is PCA anyway?

This explanation follows the reference: https://www.stat.cmu.edu/~cshalizi/uADA/12/lectures/ch18.pdf.

#### Maximum Information Retention

When you reduce the dimension of a data set you are inherently losing information. Therefore when you reduce the dimension you want to ensure that you do it in a way that "retains as much information as possible". PCA tackles this problem in a very statistical manner.

There's an idea in statistics that the information of a data set is located within that data set's variation. Thus when you reduce the dimension of a data set, you want to project your data onto a hyperplane that captures as much of the original variance in the data as possible. Thinking in terms of optimization, your goal is to project into a lower dimensional hyperplane in a way that maximizes variance.

Here's a heuristic algorithm:
1. Center your data so that each feature has 0 mean.
2. Find the direction in space along which projections have the highest variance, this produces the first principal component.
3. Find the direction orthogonal to the first principal component that maximizes variance, this is the second principal component.
4. Continue in this way, the kth principal component is the variance-maximizing direction orthogonal to the previous k-1 components.

Let's see what we mean in a 2-D example:

In [ ]:
import numpy as np

In [ ]:
rng = np.random.RandomState(1)
X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
plt.scatter(X[:, 0], X[:, 1])
plt.axis('equal');

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 2)
pca.fit(X)

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal');

Our first principal component is found along the longer arrow, and the second along the shorter arrow.

So how can we go about projecting and maximizing variance in this way?

#### Minimal Perpendicular Lengths

One way to maximize variance is to minimize the collective length of the perpendiculars from the observed data points to the fitted hyperplane. This was actually Pearson's original formulation of the problem, as his goal was to contemplate what we mean by line (or plane) of best fit. Consider this image from Pearson's 1903 paper.

<img src = "PearsonLine.png" width = "500"></img>

Recall that the Mean Square Error is given by the following:
$$
MSE = 1/n\sum_{i = 1}^{n} ||\text{observed point }i - \text{fit of point }i ||^2
$$
Where we are assuming $n$ total observations and by fit we mean the projection of observation $i$ onto the hyperplane. It can be mathematically shown that minimizing the $MSE$ is equivalent to maximizing the variance!

We will not go through the details here.

#### Eigen Data

Now from the derivation of that fact a constrained optimization problem can be set up, in which the end result shows that the principal components come from the eigenvectors of the covariance matrix of the original data. The first principal component is the eigenvector corresponding to the largest eigenvalue, the second principal component is the eigenvector corresponding to the second largest eigenvalue and so on.

This was just a quick rundown of the mathematics/statistics behind PCA. For more information please read through the two resources I have linked to above. For the rest of the PCA section we will actually implement PCA and see what it can do.

### Implementing PCA

Let's use PCA and see how powerful it can be.

#### The PC(ancer)A

We'll start by showing how PCA can help identify benign and malignant cancer using the `sklearn` cancer data set. Note, this example comes from the book, <u>Introduction to Machine Learning with Python</u>, by Andreas C. M&uuml;ller & Sarah Guido.

In [ ]:
# import the data
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
malignant = cancer.data[cancer.target == 0]
benign = cancer.data[cancer.target == 1]

In [ ]:
fig, axes = plt.subplots(15, 2, figsize = (10,20))

ax = axes.ravel()

for i in range(30):
    _, bins = np.histogram(cancer.data[:, i], bins = 50)
    ax[i].hist(malignant[:, i], bins = bins, color = 'red', alpha = .5)
    ax[i].hist(benign[:, i], bins = bins, color = 'blue', alpha = .5)
    ax[i].set_title(cancer.feature_names[i])
    ax[i].set_yticks(())

ax[0].legend(['malignant', 'benign'], loc = 'best')
    
fig.tight_layout()

As we can see there are definitely some differences between malignant and benign cancer, however, determining cutoffs or decision rules could prove troublesome with 30 different variables to consider. Let's see if PCA with 2 components can help.

Note first we have to preprocess the data a little and scale the variables, otherwise a variable like worst area will have more impact on the variance than variables like worst symmetry and worst fractal dimension.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Scaling the data, standard scaler scales the data 
# like a standard normal rv, this may not be the best way
# to scale the data though
scaler = StandardScaler()
scaler.fit(cancer.data)
X_scaled = scaler.transform(cancer.data)

In [ ]:
# Now we do PCA
pca = PCA(n_components = 2)

# Fit the PCA model to the scaled data
pca.fit(X_scaled)

# Transform the data, i.e. project onto the fitted hyperspace
X_pca = pca.transform(X_scaled)

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
# Now we plot and see what happened:
cancer_pca = pd.DataFrame({'pca_1':X_pca[:,0],'pca_2':X_pca[:,1],'cancer_outcome':cancer.target})

sns.lmplot(data = cancer_pca, x = 'pca_1', y = 'pca_2', hue = 'cancer_outcome', 
           fit_reg = False, legend = False, height = 10, aspect = 1)

plt.legend(cancer.target_names)

plt.show()

As we can see PCA does a pretty good job capturing the split between benign and malignant tumors with only 2 dimensions!

This is wonderful, but you might have a couple of questions:

1. How can I know how many dimensions are "enough"?
2. Is there a way to interpret this data in a way that I can relate back to the original problem?

We'll start to answer both of those now.

#### What does it all mean?

A clear downside of PCA is that it isn't very easy to interpret. For instance it isn't clear from our cancer plot above what from the data leads to an increase or decrease in the x and y directions. How can we interpret what separates benign and malignant tumors from this technique?

One way we can attempt to interpret the results is to look at the principal components themselves. Principal components correspond to directions in the original data, so they are combinations of the original features. In some instances we may be able to gleam some insights from the components, however, these can also be quite complex.

In [ ]:
pca_comps = pd.DataFrame({'Feature':cancer.feature_names,
                          'First Component':pca.components_[0,:],
                          'Second Component':pca.components_[1,:]})

pca_comps = pca_comps.sort_values(['Second Component'])

pca_comps

In [ ]:
# We can visualize with a heatmap
plt.matshow(pca.components_, cmap = 'viridis')

plt.yticks([0,1], ["First Component", "Second Component"],
          fontsize = 14)

plt.colorbar()

plt.xticks(range(len(cancer.feature_names)),
          cancer.feature_names, rotation = 90, 
          fontsize = 14)

plt.xlabel("Feature", fontsize = 14)
plt.ylabel("Principal Components", fontsize = 14)

plt.show()

Now let's try and interpret the results. Recall that the first component gives us the x-coordinate on our projection plot above. The first component from the PCA is positive for all features. This means that an increase in any one of the features causes an increase in the first component. As we can see from our plot this means that a large value for any one of the features is likely not a good sign, because the right half of the project plot is dominated by malignant observations.

In the second component we have some differences in signs. This can help us identify which features lead to an increase or decrease in our y-coordinate. For instance an increase in mean radius pushes our projection point down in the plot while an increase in mean fractal dimension would push our point up in the plot.

Combining our knowledge of the components with the projection plot can allow us to better understand what features of the data coincide with a malignant observation and what features coincide with a benign observation.

#### Explained Variance Ratio - How many dimensions?

How many dimensions do I need to project down to? Well that depends upon the problem at hand, but `sklearn`'s PCA function does give us the tools to figure out how much of the variance each principal component explains. This is known as the explained variance ratio of the component and is available via the explained_variance_ratio_ variable. These values indicate the proportion of the dataset's variance that lies along the axis of each principal component.

In [ ]:
print(pca.explained_variance_ratio_)

This outcome tells us that 44.27% of the variance in the original data is contained in the first component and 18.97% in the second component. Perhaps we'd like more variance, or less. We can see how much of the variance we capture in total by taking the cumulative sum of the ratios for each component.

In [ ]:
pca = PCA(n_components = 30)

pca.fit(X_scaled)

sns.set_style('whitegrid')

plt.figure(figsize = (8,6))
plt.plot(range(1,31), np.cumsum(pca.explained_variance_ratio_))
plt.axis((1,30,0,1.1))
plt.title("Explained Variance Ratio for Cancer Data",fontsize = 16)
plt.xlabel("Dimensions",fontsize = 14)
plt.ylabel("Explained Variance", fontsize = 14)
plt.show()

A nice feature of `sklearn`'s PCA is that you can tell it what you want the cumulative explained variance ratio to be, and then it will choose the minimum number of components that meat that specification.

In [ ]:
# Here's an example where we set it at .85
pca = PCA(n_components = .85)

X_pca = pca.fit_transform(X_scaled)

print(X_pca.shape)

One rule of thumb for choosing the dimension of the hyperplane is to look for the "elbow" in the cumulative explained variance plot. This is where the explained variance ratio stops growing as quickly and is thought of as the "intrinsic dimensionality" of the dataset. This looks to be at around 6 or 7 dimensions for the cancer data set.

We now have a feel for PCA. Before we move on we'll do a few more interesting examples of applying PCA.

### P(olitical)CA

Political affiliations result in a very natural classification problem. There are clearly marked political affiliations, Republican, Democrat, Independent. As it turns out we can use PCA to help take rather large datasets and project down onto two dimensions that capture political affiliation quite well.

#### Roll Call Classifications

This section was inspired from a talk given by <a href = "https://www.math.ucla.edu/~mason/">Mason Porter</a>, for reference see the two papers he has published <a href = "https://www.sciencedirect.com/science/article/pii/S0378437107007844">here</a> and <a href = "https://www.pnas.org/content/102/20/7057.short">here</a>. 

Every Congress, congressman must meet and cast votes on various bills. Congressman can vote 'Yea', 'Nay', 'Abstain' or be absent. From this information you can construct a matrix for every single session of congress where each row is a congressman and each column is a particular roll call vote. Every entry in this matrix is a $1$ if the congressman voted 'Yea', a $-1$ if the congressman voted 'Nay', and $0$ otherwise.

Below we will see what happens when we put such a matrix through PCA with 2 components.

All data was taken from <a href = "https://voteview.com/data">voteview.com</a>, and then cleaned to be in the presented format.

In [ ]:
# We'll look at the 112th Senate
congress_112 = pd.read_csv("Senate_112_roll_mat.csv")

congress_112.loc[congress_112.party_code == 200, "party_code"] = "Republican"
congress_112.loc[congress_112.party_code == 100, "party_code"] = "Democrat"
congress_112.loc[congress_112.party_code == 328, "party_code"] = "Independent"

congress_112.head()

Now the first three columns of the dataframe are identifiers so we know who cast the vote. The name is the name, the icpsr is the identifier code for that congressman in that particular congress, and the party_code is a code given for the political affiliation of the congressman (100 is Democrat, 200 is Republican, and 328 is Independent). The remaining columns each represent a bill that was voted on.

In [ ]:
total_votes = len(congress_112.columns) -3

# Retrieve Just the Votes
X = congress_112[list(map(str,range(1,total_votes + 1)))]

# Perform PCA
pca = PCA(n_components = 2)

fit = pca.fit_transform(X)

congress_112_pca = congress_112[['name','icpsr','party_code']].copy()
congress_112_pca['pca_x'] = fit[:,0]
congress_112_pca['pca_y'] = fit[:,1]

In [ ]:
sns.set_style("white")

sns.lmplot(data = congress_112_pca, 
           x = 'pca_x', 
           y = 'pca_y', 
           hue = 'party_code', 
           palette = {"Democrat":'blue', "Republican":'red', "Independent": 'purple'},
           fit_reg = False, 
           height = 7, aspect = 1, 
           scatter_kws = {'s':15})

plt.show()

Now this might not seem too impressive, way to go we recovered that there are two political parties. But think about what we did. We took information stored in 486 columns, compressed it to two columns and still retained meaningful information.

Heck we could even get away with one dimension it seems. Let's examine what it looks like with just the first principal values.

In [ ]:
congress_112_pca['Dummy'] = ""

sns.stripplot(data = congress_112_pca, 
              x = 'pca_x',
              y = 'Dummy',
              hue = 'party_code',
              palette = {"Democrat":'blue', "Republican":'red', "Independent": 'purple'}, 
              jitter = 0)

plt.ylabel("")
plt.show()

While the sole independent does get swallowed up here, we see that we only need one dimension to explain the party split. There really isn't much variation in the way our congressman vote. 

A fun fact, the 2-D PCA projection is how voteview.com calculates various ideological scores for Congress members. They then use those scores to make probabilstic predictions on how a congressman will vote on a particular bill. Check out voteview.com for more info!

We could also investigate further. For instance all of the Democrats are much more closer than the Republicans is there a reason for that? There's a cluster of three republicans in the upper right hand corner of the 2-D plot, is there a reason they're further away from the main Republican grouping?

##### Practice

Now you take an opportunity to practice. In the repository are data files for the 113th, 114th, and 115th Senates. Run PCA on those sets and see what you get. Is the output the same as 112th or are there differences over time?

In [ ]:
# csvs are stored as Senate_#_roll_mat.csv







#### Twitter Polarization

A great fear of social media is that it is leading to greater polarization of our society, but what does this have to do with PCA?

Well lets look at a data set collected by the Center for the Study of Networks and Society (CSNS). For a <a href = "http://www.susanbourbaki.com/category/breitbart/">blog post</a> back in 2017 in which they examined the following habits of various Twitter users they collected every follower from the following 30 accounts: DrDavidDuke, AryanBrother, CofCC76, Hatchet318, KKKOfficial311, KeyStoneUnited, MatthewHeimbach, ThaRightStuff, nsm88, BreitbartNews, FiveThirtyEight, csmonitor, MotherJones, NRO, dailykos, theblaze, thenation, DRUDGE_REPORT, WSJ, washingtonpost, FoxNews, NPR, realDonaldTrump, tedcruz, BernieSanders, SenWarren, SpeakerRyan, SenJohnMcCain, marcorubio, and CoryBooker.

Now we will depart from the story told by CSNS and look at a specific subset of their data, just those followers of the 13 media accounts: BreitbartNews, FiveThirtyEight, csmonitor, MotherJones, NRO, dailykos, theblaze, thenation, DRUDGE_REPORT, WSJ, washingtonpost, FoxNews, NPR. However, I encourage you to read their blog post as well as their recent paper on arxiv, <a href = "https://arxiv.org/pdf/1905.07755.pdf"><i>Online reactions to the 2017 ‘Unite the Right’ rally in Charlottesville: Measuring polarization in Twitter networks using media followership</i></a> that includes a more in depth analysis of this data in the context of the Charlottesville incident.

Our goal will be to see if PCA can illuminate any information on the anonymized Twitter users based on their media account followership. One more additional caveat before we move forward, the original media restricted dataset contains 27,056,206 unique Twitter accounts. We will only look at a stratified sample of this data that contains only 100,000 accounts. The data has been sampled in such a way that we retain the results found by CSNS.

In [ ]:
# Load the data
twitter = pd.read_csv("twitter_sba.csv")

twitter.head()

Now before performing PCA let's look at the believed political leanings of each of our media accounts.

| Twitter Screenname | News Source               | Leaning Tendency |
|--------------------|---------------------------|------------------|
| FoxNews            | Fox News                  | Right            |
| BreitbartNews      | Breitbart                 | Right            |
| DRUDGE_REPORT      | The Drudge Report         | Right            |
| the blaze          | TheBlaze                  | Right            |
| NRO                | National Review           | Right            |
| WSJ                | Wall Street Journal       | center           |
| csmonitor          | Christian Science Monitor | left             |
| FiveThirtyEight    | FiveThirtyEight           | left             |
| dailykos           | Daily KOS                 | left             |
| thenation          | The Nation                | left             |
| MotherJones        | Mother Jones              | left             |
| washingtonpost     | The Washington Post       | left             |
| NPR                | NPR                       | left             |

These tendencies have been corroborated by peer reviewed studies. The Wall Street Journal is labeled as center because it is widely considered to be conservative-leaning, but has been grouped as liberal based on readership and co-citations.

#### Practice
Now perform PCA and see what we get. Use 3 components.

In [ ]:
# Do your work here














### PCA Isn't Always the Answer

As we will see in the following section PCA isn't the best choice for every data compression task. Although it can be quite powerful.

## t-Distributed Stochastic Neighbor Embedding (t-SNE)

One reason we're interested in dimensionality reduction is so we can better visualize what is going on in a high dimensional data set. As we have said before, we are 3-D creatures and so our heads can really only wrap around 2, 3 or maybe 4 dimensions. Thus it is key to have algorithms at your disposal that allow you to visualize high dimensional data in low dimensions.

We'll start with an example to show how good t-SNE is at this.

### Back to MNIST

Recall our MNIST data that we stored in `digits`.

We'll first visualize that data using PCA and then compare it to the visualization from t-SNE.

In [ ]:
pca.fit(digits.data)
pca_digits = pca.transform(digits.data)

pca_digits_df = pd.DataFrame({"Digit":digits.target,
                              "pca_x":pca_digits[:,0],
                              "pca_y":pca_digits[:,1]})

In [ ]:
plt.figure(figsize = (14,12))

for target in list(set(digits.target)):
    temp = pca_digits_df.loc[pca_digits_df.Digit == target,]
    plt.plot(temp.pca_x, temp.pca_y, '.', label = target, markersize = 10)
    
plt.legend(fontsize = 14)
plt.show()

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components = 2, random_state = 440)

In [ ]:
tsne_digits = tsne.fit_transform(digits.data)

tsne_df = pd.DataFrame({"Digit":digits.target,
                        "tsne_x":tsne_digits[:,0],
                        "tsne_y":tsne_digits[:,1]})

In [ ]:
plt.figure(figsize = (14,10))

for target in list(set(digits.target)):
    temp = tsne_df.loc[tsne_df.Digit == target,]
    plt.plot(temp.tsne_x, temp.tsne_y, '.', label = target, markersize = 10)
    
plt.legend(fontsize = 14)
plt.show()

We can see that, while not perfect, the t-SNE plot does a significantly better job of separating out the clusters of digits in the dataset. That's because the algorithm was designed with that in mind.


### How Does t-SNE Work?

We'll give a brief overview of how the algorithm works here. For the rigorous treatment see <a href = "http://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf"><i>Visualizing Data using t-SNE</i></a> by Laurens van der Maaten and Geoffrey Hinton.

Here's a basic outline of the algorithm:
1. For all points $x_i$, $x_j$, convert their Euclidean distance into a conditional probability $p_{j|i}$. This is done by imagining a Gaussian distribution around $x_i$ and then comparing the "normal distance" of $x_j$ vs the sum of all other "normal distances". Here's a precise mathematical formula:
$$
p_{j|i} = \frac{\exp(-||x_i - x_j||^2/2\sigma_i^2)}{\sum_{k\neq i} \exp(-||x_i - x_k||^2/2\sigma_i^2)}
$$
Think of $p_{j|i}$ as the probability that $x_i$ would choose $x_j$ as its neighbor. We take $p_{i|i} = 0$.
2. For every point $x_i$ in high dimensional space we will have a low dimensional counterpart, $y_i$, to which we map $x_i$. Similar to $p_{j|i}$ we will have $q_{j|i}$ that gives the probability that $y_i$ would choose $y_j$ as a neighbor. Here is the precise mathematical formula they are using:
$$
q_{j|i} = \frac{(1 + ||y_i - y_j||^2)^{-1}}{\sum_{k\neq l} (1 + ||y_k - y_l||^2)^{-1}}
$$
We take $q_{i,i} = 0$. Note: this is where the $t$ comes from! The numerator and denominator of $q_{j|i}$ is from the probability density for the $t$ distribution with 1 degree of freedom.
3. Now if we're preserving these pairwise distances well, then we should expect $p_{j|i}$ to be close to $q_{j|i}$. In order to do this a cost function that measures the difference between $p_{j|i}$ and $q_{j|i}$ is minimized using gradient descent. The optimal $y_i$s are then spit out by the algorithm.


You may be wondering how $\sigma_i$ is chosen. The idea behind letting $\sigma_i$ vary with $i$ is that some regions of the data are much denser than others. Changing $\sigma_i$ with $i$ allows the algorithm to essentially control the number of neighbors of $x_i$ it considers. We can control this in $t-SNE$ by specifying a perplexity. Typical values for perplexity are in 5 to 50.

If you'd like a more in depth non research paper explanation of t-SNE check out this blog post, http://mlexplained.com/2018/09/14/paper-dissected-visualizing-data-using-t-sne-explained/. I think the author does a great job explaining it.


#### Elevator Pitch for t-SNE

Okay so maybe that was a lot of probability theory, even if it was watered down from the paper. Here's an elevator pitch. PCA is typically good at recognizing global structures of the data. t-SNE is able to detect local structures, in particular it is great at detecting clusters in the data as we saw in digits example.

If you are working on a classification problem it might be a good idea to look at your data in lower dimensions in t-SNE. 

Let's see another example contrasting PCA and t-SNE.

#### A Blob in a Blob

This will be an artificial data set of a 3-D data blobs. We'll see how bad PCA is at detecting the local structure and then contrast that performance with t-SNE. This example is from the aforementioned t-SNE blog, http://mlexplained.com/2018/09/14/paper-dissected-visualizing-data-using-t-sne-explained/.

In [ ]:
import numpy as np

In [ ]:
# First blob
A = np.random.normal(scale = 1, size = (300,3))

# The blob that surrounds A
B = np.array([x for x in np.random.normal(scale = 5, size = (1000,3)) if np.linalg.norm(x) > 7])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = Axes3D(fig)

ax.scatter(A[:,0], A[:,1], A[:,2])
ax.scatter(B[:,0], B[:,1], B[:,2])

plt.show()

In [ ]:
# Combine all the data
X = np.r_[A,B]

In [ ]:
# First PCA
X_PCA = PCA(n_components = 2).fit_transform(X)

A_PCA = X_PCA[:A.shape[0],:]
B_PCA = X_PCA[A.shape[0]:,:]

plt.plot(A_PCA[:,0], A_PCA[:,1], '.')
plt.plot(B_PCA[:,0], B_PCA[:,1], '.')

plt.show()

Now if we didn't know that there were two blobs of data ahead of time, PCA wouldn't give us that structure. Let's examine the t_SNE now.

In [ ]:
X_tsne = TSNE(n_components = 2, perplexity = 20).fit_transform(X)

A_tsne = X_tsne[:A.shape[0],:]
B_tsne = X_tsne[A.shape[0]:,:]

plt.plot(A_tsne[:,0], A_tsne[:,1], '.')
plt.plot(B_tsne[:,0], B_tsne[:,1], '.')

plt.show()

#### Practice

Play around with the perplexity above, what happens as you make it smaller, larger?

#### Movie Genres

From the following kaggle data set, https://www.kaggle.com/carolzhangdc/imdb-5000-movie-dataset/downloads/imdb-5000-movie-dataset.zip/1, I extracted the movie name and the genres. From that data I created the following dataframe.

In [ ]:
movies = pd.read_csv("movies.csv")

In [ ]:
movies.head()

There are 26 possible genres, and for each genre column there is a 0 or 1 depending on whether or not that movie was classified as that genre.

Using t-SNE explore if there are natural genre groupings that many films fall into. You could look at single genres like Documentary, or combinations like Fantasy Adventure. Consider adding a binary variable for a film series like Star Wars, do those films go to the same place in the plot? 

In [ ]:
# Code here











In [ ]:
# Also code here










In [ ]:
# Maybe you need three spaces








### Warnings About t-SNE

1. The S stands for Stochastic, meaning that your results change slightly each time.

In [ ]:
# Run this 5 times
X_tsne = TSNE(n_components = 2, perplexity = 20).fit_transform(X)

A_tsne = X_tsne[:A.shape[0],:]
B_tsne = X_tsne[A.shape[0]:,:]

plt.plot(A_tsne[:,0], A_tsne[:,1], '.')
plt.plot(B_tsne[:,0], B_tsne[:,1], '.')

plt.show()

2. You can't really use this to make predictions on new data, unlike PCA there isn't a procedure that will map new points into the lower dimensional space. 

3. The magnitude of the distances between clusters shouldn't be interpreted. They are mostly a result of the t distribution.

4. t-SNE results should not be used as statistical evidence or proof of something. It is not a formal statistical test.

5. Use t-SNE to explore data and make hypotheses as well as cool plots!

## Wrapping Up

In this lecture we went through PCA and t-SNE. Now these are not the only two dimensionality reduction techniques, there are many more. They are two of the most popular techniques in data science so you're off to a good start.

You can even combine these two techniques. Often for data with 1000s of features people will use PCA to reduce the data to the intrinsic dimensionality of the data, and then explore using t-SNE.